In [150]:
%matplotlib inline

import os
import sys
import random
import imp

from IPython import display


import numpy as np
import pandas as pd
import xarray as xr


import climatools.aerosol.aerosol as climaaerosol
import climatools.io.cesm as climaio
import climatools.aerosol.aerosol_constants as aeroconst
import climatools.rrtmg.column_model as rrtmg_column
import climatools.units as climaunits


imp.reload(climaaerosol)
imp.reload(climaio)
imp.reload(aeroconst)
imp.reload(rrtmg_column)
imp.reload(climaunits)

<module 'climatools.units' from '/nuwa_cluster/home/jackyu/climatools/climatools/units.py'>

### General workflow

In [131]:
## where are the CAM history files
dir_camhist = '/scratch/lupin/cesm1/archive/f19_g16.B1850.std/atm/hist'

In [132]:
## Prep o3 concentration data

ds_o3 = climaaerosol.OZONE_DATASET
da_o3 = ds_o3['O3']

# interpolate o3 concentration along lev
levinterpfunc_da_o3 = climaaerosol.get_interpfunc(da_o3, dim='lev')

# evaluate o3 concentration at the 30 levs of CAM history
filepath_camhist = os.path.join(dir_camhist,
                                random.choice(os.listdir(dir_camhist)))
ds_camhist = xr.open_dataset(filepath_camhist, decode_cf=False)

da_o3_levinterp = levinterpfunc_da_o3(coords=ds_camhist.coords['lev'])


# interpolate o3 concentration along time
timeinterpfunc_o3 = climaaerosol.get_interpfunc(da_o3_levinterp,
                                                dim='time')

In [133]:
# use this cell if IATM = 1 (i.e using RRTATM) but currently this raises an error, need to 
# provide altitude and pressure very carefully


aerosol = False

print("load a CAM history file from some previous global run")
ds = climaio.load_camhistory(readfrom='f19_g16.B1850.std.cam2.h0.0001-06.nc')

print("convert hybrid levels/layers to pressure levels/layers")
ds = climaunits.hybrid2mbar(ds)

print("get ozone concentration for the time in this CAM history file")
ds = climaaerosol.get_o3_concentration(ds, interpfunc=timeinterpfunc_o3)

print("get o2 concentration")
ds = climaaerosol.get_o2_concentration(ds)

print("interpolate layer values to levels for: T, Q and O3")
ds = climaaerosol.interp_layers2levels(ds, vars=['T', 'Q', 'O3', 'Z3'])

if aerosol:
    print('You asked for aerosol. You got it.')
    # gather aerosol species mmr into a new variable in the dataset
    ds = climaaerosol.aerosol_species_mmr(ds)

    # aerosol water uptake. computes qaerwat, dgncur_awet, wetdens. add to ds
    ds = climaaerosol.wateruptake(ds)
    
    # get pressure difference for layer mass
    ds = climaaerosol.get_pressure_difference(ds)

    # compute and add to ds modal aerosol optical properties 
    ds = climaaerosol.modal_aero_sw(ds)

stackdims = ('time', 'lat', 'lon')


#for time, lat, lon in cols.values[:1]:
#    print("Working on column at time {}, lat {}, lon {}".format(time, lat, lon))
#    print("write input tapes for RRTMG column model")
#    rrtmg_column.write_sw_inputfiles(ds,
#                                     time=time, lat=lat, lon=lon,
#                                     aerosol=aerosol)

#    print("execute RRTMG")
#    assert os.system('./rrtmg.exe') == 0

#    print("this dataset contains things like flux down, up and heating rates")
#    ds_rrtmg_col = climarrtmg.load_OUTPUT_RRTM(readfrom='OUTPUT_RRTM', 
#                                               signed_fluxes=True)
#    ds_rrtmg.loc[dict(time=time, lat=lat, lon=lon)] = ds_rrtmg_col

#    print("write RRTMG output to netCDF file")
#    ds_rrtmg.to_netcdf('RRTMG_f19_g16.B1850.std.cam2.h0.0001-06.nc')
    
#    print()




load a CAM history file from some previous global run
convert hybrid levels/layers to pressure levels/layers
get ozone concentration for the time in this CAM history file
get o2 concentration
interpolate layer values to levels for: T, Q and O3


In [152]:
# select a single (time, lat, lon)
ds_col = ds.sel(time=0, lat=90, lon=0, method='nearest')

In [153]:
# keep only the needed data variables to begin with
ds_col = ds_col[['ilev', 'lev',
                 'level_pressure', 'layer_pressure', 'PS',
                 'iZ3', 'Z3',
                 'iT', 'T',
                 'iQ', 'Q', 
                 'co2vmr', 
                 'iO3', 'O3', 
                 'n2ovmr', 'ch4vmr', 'o2mmr']]

In [154]:
ds_col = ds_col.rename({'T': 'layer_temperature',
                        'iT': 'level_temperature'})

In [155]:
ds_col = ds_col.rename({'Z3': 'layer_altitude',
                        'iZ3': 'level_altitude'})

In [156]:
# H2O: rename iQ level_mmr_h2o
ds_col = ds_col.rename({'iQ': 'level_mmr_h2o'})

In [157]:
# CO2: broadcast co2vmr to all levels and name it level_vmr_co2
level_vmr_co2 = (ds_col['co2vmr'].values 
                 * np.ones((ds_col.dims['ilev'])))
ds_col['level_vmr_co2'] = (('ilev',),
                           level_vmr_co2,
                           {'units': 'l/l',
                            'long_name': 
                            'co2 mixing ratio by volume'})

In [158]:
# O3: rename iO3 to level_vmr_o3
level_vmr_o3 = (ds_col['iO3'].values 
                * np.ones((ds_col.dims['ilev'])))
ds_col['level_vmr_o3'] = (('ilev', ),
                          level_vmr_o3,
                          {'units': 'l/l',
                           'long_name':
                           'o3 mixing ratio by volume'})

In [159]:
# N2O: broadcast n2ovmr to levels and name it level_vmr_n2o
level_vmr_n2o = (ds_col['n2ovmr'].values 
                 * np.ones((ds_col.dims['ilev'])))
ds_col['level_vmr_n2o'] = (('ilev',),
                           level_vmr_n2o,
                           {'units': 'l/l',
                            'long_name': 
                            'n2o mixing ratio by volume'})


In [160]:
# CO: set to zero at all levels
ds_col['level_vmr_co'] = (('ilev', ),
                          np.zeros((ds_col.dims['ilev'])),
                          {'units': 'l/l', 
                           'long_name':
                           'co mixing ratio by volume'})

In [161]:
# CH4: broadcast ch4vmr to all levels and name it level_vmr_ch4
level_vmr_ch4 = (ds_col['ch4vmr'].values 
                 * np.ones((ds_col.dims['ilev'])))
ds_col['level_vmr_ch4'] = (('ilev',),
                           level_vmr_ch4,
                           {'units': 'l/l',
                            'long_name':
                            'ch4 mixing ratio by volume'})

In [162]:
# O2: broadcast o2mmr to all levels and name level_mmr_o2
level_mmr_o2 = (ds_col['o2mmr'].values
                * np.ones((ds_col.dims['ilev'])))
ds_col['level_mmr_o2'] = (('ilev',),
                          level_mmr_o2,
                          {'units': 'l/l',
                           'long_name': 
                           'o2 mixing ratio by mass'})

In [163]:
ds_col['juldat'] = ((), 0., 
                    {'long_name':
                     'Julian day associated with calculation',
                     'units': 'days'})

ds_col['sza'] = ((), 60,
                 {'long_name':
                  'solar zenith angle in degrees',
                  'units': 'degrees'})

In [164]:
ds_col = ds_col.drop(['Q', 'co2vmr', 'O3', 
                      'n2ovmr', 'ch4vmr', 'o2mmr'])

In [165]:
rrtmg_column.write_input_rrtm(ds=ds_col, aerosol=False, iatm=1)

In [151]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')